In [1]:
from modelDesign import *

2024-08-29 20:37:06.560907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 20:37:06.600523: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 20:37:06.611674: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 20:37:06.659321: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 20:37:07.428510: W tensorflow/compiler/tf2

In [2]:
df = pd.read_csv('data/player_stats.csv')
df = df.sort_values(by=['player_id', 'season', 'week'])

## Data Preparation

In [3]:
sequence_length = 3
tscv = TimeSeriesSplit(n_splits=5)

df = featureEngineering(df)

qb_df_features, qb_df_target, rb_df_features, rb_df_target, wr_df_features, wr_df_target, te_df_features, te_df_target = cleanData(df)

X_qb, y_qb = create_sequences(qb_df_features.values, qb_df_target.values, sequence_length)
X_rb, y_rb = create_sequences(rb_df_features.values, rb_df_target.values, sequence_length)
X_wr, y_wr = create_sequences(wr_df_features.values, wr_df_target.values, sequence_length)
X_te, y_te = create_sequences(te_df_features.values, te_df_target.values, sequence_length)

/home/calder/Documents/ff2024/dataPreparation.py:31: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[numeric_cols] = df[numeric_cols].applymap(lambda x: np.nan if np.isinf(x) else x)
/home/calder/Documents/ff2024/dataPreparation.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(


In [4]:
datasets = {
    'qb': [],
    'rb': [],
    'wr': [],
    'te': []
}

for X, y, key in [(X_qb, y_qb, 'qb'), 
                  (X_rb, y_rb, 'rb'), 
                  (X_wr, y_wr, 'wr'), 
                  (X_te, y_te, 'te')]:
    for i, (X_train, X_test, y_train, y_test) in enumerate(process_data(X, y, tscv)):
        datasets[key].append({
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test
        })

# Training/Hyperparameter tuning

In [5]:
import keras_tuner as kt

# Instantiate the tuner
tuner = kt.RandomSearch(
    buildModelGRU,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=3,
    directory='tuning',
    project_name='gru_tuning'
)

# Perform hyperparameter tuning
tuner.search(X_train, y_train, validation_data=(x_val, y_val), epochs=50)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

I0000 00:00:1724978533.531052  411579 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724978533.603134  411579 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724978533.606403  411579 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724978533.609505  411579 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

ValueError: Cannot convert '<keras_tuner.src.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7e10a9b422c0>' to a shape.

In [6]:
early_stopping = EarlyStopping(
    monitor='val_loss',        # Metric to monitor
    patience=10,                # Number of epochs with no improvement to wait before stopping
    restore_best_weights=True  # Restore model weights from the epoch with the best metric value
)

# Training

### GRU

In [7]:
# run the model on the normalized datasets
rnn_models_gru = {
    'qb': [],
    'rb': [],
    'wr': [],
    'te': []
}

for key, dataset in datasets.items():
    for i, dataset in enumerate(dataset):
        print(f"Training RNN model for {key} - split {i + 1}...")
        
        # Get the input shape
        input_shape = dataset['X_train'].shape[1:]
        
        # Create the RNN model
        model = buildModelGRU(input_shape,
                                units=512
                                )
        
        # Train the model
        model.fit(
            dataset['X_train'],
            dataset['y_train'],
            epochs=15,
            batch_size=32,
            validation_data=(dataset['X_test'], dataset['y_test']),
            verbose=1,
            callbacks=[early_stopping]
        )
        
        # Store the model
        rnn_models_gru[key].append(model)

Training RNN model for qb - split 1...
Epoch 1/150


2024-08-29 20:51:35.431984: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


81/81 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 190.8867 - mae: 11.5195 - val_loss: 200.5915 - val_mae: 11.6470
Epoch 2/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 157.7991 - mae: 10.5029 - val_loss: 174.0139 - val_mae: 10.5149
Epoch 3/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 116.0789 - mae: 8.8017 - val_loss: 126.9804 - val_mae: 8.7046
Epoch 4/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 81.6059 - mae: 7.1240 - val_loss: 110.4215 - val_mae: 8.0904
Epoch 5/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 62.4798 - mae: 6.2142 - val_loss: 93.9928 - val_mae: 7.5763
Epoch 6/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 53.1289 - mae: 5.8489 - val_loss: 101.2048 - val_mae: 7.9349
Epoch 7/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 51.9583 - mae: 5.7183 - val_loss: 94.5967 - val_mae: 7.7643
Epoch 8/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 46.3115 - mae: 5.4249 - val_loss: 91.1161 - val_mae: 7.5595
Epoch 9/150
81/81 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/s

2024-08-29 20:56:30.728364: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 500385600 exceeds 10% of free system memory.
2024-08-29 20:56:31.161811: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 500385600 exceeds 10% of free system memory.


Epoch 1/150
322/322 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 178.3975 - mae: 11.1889 - val_loss: 64.2868 - val_mae: 6.2758
Epoch 2/150
322/322 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 62.3782 - mae: 6.2362 - val_loss: 61.5848 - val_mae: 6.3263
Epoch 3/150
322/322 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 57.5633 - mae: 6.0753 - val_loss: 59.5137 - val_mae: 6.2207
Epoch 4/150
322/322 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 54.3231 - mae: 5.8382 - val_loss: 61.1657 - val_mae: 6.2971
Epoch 5/150
322/322 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 52.4668 - mae: 5.7578 - val_loss: 59.6603 - val_mae: 6.2067
Epoch 6/150
322/322 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 51.2449 - mae: 5.7028 - val_loss: 60.3531 - val_mae: 6.2313
Epoch 7/150
322/322 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 47.4634 - mae: 5.4488 - val_loss: 63.0276 - val_mae: 6.3662
Epoch 8/150
322/322 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 43.3670 - mae: 5.2166 - val_loss: 65.2198 - val_mae: 6.4130
Epoch 9/150
322/322 ━━━━━━━━━━

2024-08-29 20:59:18.655577: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 625482000 exceeds 10% of free system memory.
2024-08-29 20:59:19.202885: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 625482000 exceeds 10% of free system memory.


Epoch 1/150
403/403 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 170.2933 - mae: 10.8467 - val_loss: 68.1674 - val_mae: 6.4803
Epoch 2/150
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 58.9606 - mae: 6.1122 - val_loss: 68.3862 - val_mae: 6.6136
Epoch 3/150
403/403 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 57.3789 - mae: 6.0633 - val_loss: 69.4232 - val_mae: 6.6230
Epoch 4/150
403/403 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 55.3848 - mae: 5.9535 - val_loss: 66.6794 - val_mae: 6.5311
Epoch 5/150
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 52.5764 - mae: 5.7933 - val_loss: 71.3637 - val_mae: 6.7509
Epoch 6/150
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 49.4933 - mae: 5.6410 - val_loss: 73.0152 - val_mae: 6.8195
Epoch 7/150
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 46.6431 - mae: 5.4500 - val_loss: 74.0532 - val_mae: 6.9085
Epoch 8/150
403/403 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 42.7191 - mae: 5.1964 - val_loss: 77.8242 - val_mae: 7.0435
Epoch 9/150
403/403 ━━━━━━━━━━

# Model tuning and cross-validation

# Testing

### GRU

In [ ]:
# Test the model on the test data
rnn_predictions_gru = {
    'qb': [],
    'rb': [],
    'wr': [],
    'te': []
}

for key, models in rnn_models_gru.items():
    for i, model in enumerate(models):
        dataset = datasets[key][i]
        print(f"Testing RNN model for {key} - split {i + 1}...")
        
        # Make predictions
        y_pred = model.predict(dataset['X_test'])
        
        # Store the predictions
        rnn_predictions_gru[key].append(y_pred)

# Output the predictions
for key, predictions in rnn_predictions_gru.items():
    print(f"Predictions for {key}:")
    for i, prediction in enumerate(predictions):
        print(f"  Split {i + 1}:")
        print(f"    Shape: {prediction.shape}")
        

TypeError: list indices must be integers or slices, not str

In [ ]:
type(datasets)

list

In [ ]:
rnn_models_gru

{'qb': [<Sequential name=sequential_1, built=True>,
  <Sequential name=sequential_2, built=True>,
  <Sequential name=sequential_3, built=True>,
  <Sequential name=sequential_4, built=True>,
  <Sequential name=sequential_5, built=True>],
 'rb': [<Sequential name=sequential_6, built=True>,
  <Sequential name=sequential_7, built=True>,
  <Sequential name=sequential_8, built=True>,
  <Sequential name=sequential_9, built=True>,
  <Sequential name=sequential_10, built=True>],
 'wr': [<Sequential name=sequential_11, built=True>,
  <Sequential name=sequential_12, built=True>,
  <Sequential name=sequential_13, built=True>,
  <Sequential name=sequential_14, built=True>,
  <Sequential name=sequential_15, built=True>],
 'te': [<Sequential name=sequential_16, built=True>,
  <Sequential name=sequential_17, built=True>,
  <Sequential name=sequential_18, built=True>,
  <Sequential name=sequential_19, built=True>,
  <Sequential name=sequential_20, built=True>]}

# Performance evaluation

#### RMSE
#### MAE

# Prediction

In [ ]:
def compute_permutation_importance(model, X_test, y_test, n_repeats=5):
    baseline_mse = mean_squared_error(y_test, model.predict(X_test))
    importances = np.zeros(X_test.shape[1])

    for i in range(X_test.shape[1]):
        shuffled_mses = []
        for _ in range(n_repeats):
            X_test_permuted = X_test.copy()
            np.random.shuffle(X_test_permuted[:, i])
            shuffled_mses.append(mean_squared_error(y_test, model.predict(X_test_permuted)))
        
        importances[i] = np.mean(shuffled_mses) - baseline_mse

    return importances

# Compute permutation feature importance for each model and each split
feature_importance_results = {
    'qb': [],
    'rb': [],
    'wr': [],
    'te': []
}

for key, models in rnn_models_gru.items():
    for i, model in enumerate(models):
        dataset = normalized_datasets[key][i]
        X_test = dataset['X_test']
        y_test = dataset['y_test']

        print(f"Computing feature importance for {key} - split {i + 1}...")
        importances = compute_permutation_importance(model, X_test, y_test)
        feature_importance_results[key].append(importances)

# Convert feature importance results to DataFrame
import pandas as pd

# Assuming all splits have the same number of features, create a DataFrame
feature_names = [f'Feature_{j}' for j in range(X_test.shape[1])]
importance_df = pd.DataFrame(np.mean(np.array([np.array(importance).T for importance in feature_importance_results[key]]), axis=0),
                             columns=feature_names)

# Plot the feature importances as a heatmap
pl.figure(figsize=(12, 8))
sns.heatmap(importance_df, annot=True, cmap='viridis', fmt=".3f", cbar=True)

pl.xlabel('Features')
pl.ylabel('Position')
pl.title('Permutation Feature Importance Heatmap')
pl.show()

Computing feature importance for qb - split 1...
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 937us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 909us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 924us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 985us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 824us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 960us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 923us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 955us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 956us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 867us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 954us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step
Computing feature importance for qb - split 2...
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 900us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 913us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 872us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 997us/step
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step
61/61 

ValueError: Shape of passed values is (3, 1), indices imply (3, 3)